## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [833]:
from pandas import *
import os
acteurs = []
base_path = '../03-corpus/2-data/1-fr/'
for file in os.listdir(base_path):
    if file.endswith('.csv'):
        acteurs.append({'acteur':file[:-4], 'status': 'to-do'})

for i in range(25):
    acteurs[i]['status'] = 'done'
DataFrame(acteurs)

,acteur,status
0,chum,done
1,chuqc,done
2,chusj,done
3,cisss_ca,done
4,cisss_cotenord,done
5,cisss_gaspesie,done
6,cisss_iles,done
7,cisss_lanaudiere,done
8,ciusss_at,done
9,ciusss_centreouest,done


In [834]:
path = '../03-corpus/2-data/1-fr/'
acteur = acteurs[i+1]['acteur']

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=';')[["Address", 'Title', 'Type', "text"]]

csv

,Address,Title,Type,text
0,https:////www.quebec.ca//sante,Santé | Gouvernement du Québec,text//html,English Nous joindre Accueil Santé Santé Alime...
1,https:////www.quebec.ca//,Bienvenue sur Québec.ca | Gouvernement du Québec,text//html,English Nous joindre Gouvernement du Québec In...
2,https:////www.quebec.ca//nous-joindre//renseig...,Renseignements généraux | Gouvernement du Québec,text//html,English Nous joindre Accueil Nous joindre Rens...
3,https:////www.quebec.ca//sante//problemes-de-s...,La maladie à coronavirus (COVID-19) au Québec ...,text//html,English Nous joindre Accueil Santé Problèmes d...
4,https:////www.quebec.ca//sante//alimentation,Alimentation | Gouvernement du Québec,text//html,English Nous joindre Accueil Santé Alimentatio...
...,...,...,...,...
1393,https:////www.quebec.ca//zh-cn//%E6%96%B0%E9%9...,NaN,NaN,Nous joindre Accueil Nouvelles Un coup d’accél...
1394,https:////www.quebec.ca//zh-hk//%E6%96%B0%E8%8...,NaN,NaN,Nous joindre Accueil Nouvelles Un coup d’accél...
1395,https:////www.quebec.ca//nl//nieuws//coup-acce...,NaN,NaN,Nous joindre Accueil Nouvelles Un coup d’accél...
1396,https:////www.quebec.ca//uk//novini//coup-acce...,NaN,NaN,Nous joindre Accueil Nouvelles Un coup d’accél...


In [835]:
liste = [lien.replace('//', "/") for lien in csv['Address'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien and not 'nouvelles' in lien]
print(str(len(liste)) + ' liens.')
liste 

1332 liens.


['https://www.quebec.ca/sante',
 'https://www.quebec.ca/',
 'https://www.quebec.ca/sante/problemes-de-sante/a-z/coronavirus-2019',
 'https://www.quebec.ca/sante/alimentation',
 'https://www.quebec.ca/sante/conseils-et-prevention',
 'https://www.quebec.ca/sante/don-de-sang-de-tissus-et-d-organes',
 'https://www.quebec.ca/sante/problemes-de-sante',
 'https://www.quebec.ca/sante/conseils-et-prevention/sante-mentale',
 'https://www.quebec.ca/sante/systeme-et-services-de-sante',
 'https://www.quebec.ca/sante/trouver-une-ressource',
 'https://www.quebec.ca/sante/vos-informations-de-sante',
 'https://www.quebec.ca/informations-et-services',
 'https://www.quebec.ca/agriculture-environnement-et-ressources-naturelles',
 'https://www.quebec.ca/culture',
 'https://www.quebec.ca/education',
 'https://www.quebec.ca/emploi',
 'https://www.quebec.ca/entreprises-et-travailleurs-autonomes',
 'https://www.quebec.ca/famille-et-soutien-aux-personnes',
 'https://www.quebec.ca/finances-impots-et-taxes',
 'ht

In [836]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens = []
for i in range (2, 9): # On prend les liens de niveau 2 à 8
    try:
        liens += [lien for lien in liste if len(lien.split('/')) > i]
    except Exception as e:
        True

In [837]:
regroupements = {}

for lien in liens:
    for i in range(3,9):
        try:
            tag = lien.split('/')[i-1]
            if tag in regroupements:
                regroupements[tag].append(lien)
                
            else:
                if len(tag)>1:
                    regroupements[tag] = []

        except Exception as e:
            True

tags = list(regroupements.keys())

In [844]:
tags = [tag for tag in regroupements if len(regroupements[tag]) > 40]

In [845]:
tags

['www.quebec.ca',
 'sante',
 'problemes-de-sante',
 'a-z',
 'coronavirus-2019',
 'alimentation',
 'conseils-et-prevention',
 'sante-mentale',
 'systeme-et-services-de-sante',
 'trouver-une-ressource',
 'agriculture-environnement-et-ressources-naturelles',
 'culture',
 'education',
 'emploi',
 'entreprises-et-travailleurs-autonomes',
 'famille-et-soutien-aux-personnes',
 'habitation-et-logement',
 'immigration',
 'justice-et-etat-civil',
 'securite-situations-urgence',
 'tourisme-et-loisirs',
 'transports',
 'gouvernement',
 'ministeres-et-organismes',
 'gouvernement-ouvert',
 'travailler-gouvernement',
 'acces-information',
 'conseils-en-recherche-demploi',
 'aide-sociale-et-solidarite-sociale',
 'politiques-orientations',
 'aide-financiere-aux-etudes',
 'metiers-professions',
 'itss',
 'informer-sur-troubles-mentaux',
 'organisation-des-services',
 'agriculture',
 'agriculture-urbaine',
 'cegep',
 'prets-bourses-temps-plein',
 'aide-financiere',
 'enfance',
 'etudier-quebec',
 'refugi

In [846]:
print('Ça nous donne {} tags.'.format(len(tags)))
tags

Ça nous donne 84 tags.


['www.quebec.ca',
 'sante',
 'problemes-de-sante',
 'a-z',
 'coronavirus-2019',
 'alimentation',
 'conseils-et-prevention',
 'sante-mentale',
 'systeme-et-services-de-sante',
 'trouver-une-ressource',
 'agriculture-environnement-et-ressources-naturelles',
 'culture',
 'education',
 'emploi',
 'entreprises-et-travailleurs-autonomes',
 'famille-et-soutien-aux-personnes',
 'habitation-et-logement',
 'immigration',
 'justice-et-etat-civil',
 'securite-situations-urgence',
 'tourisme-et-loisirs',
 'transports',
 'gouvernement',
 'ministeres-et-organismes',
 'gouvernement-ouvert',
 'travailler-gouvernement',
 'acces-information',
 'conseils-en-recherche-demploi',
 'aide-sociale-et-solidarite-sociale',
 'politiques-orientations',
 'aide-financiere-aux-etudes',
 'metiers-professions',
 'itss',
 'informer-sur-troubles-mentaux',
 'organisation-des-services',
 'agriculture',
 'agriculture-urbaine',
 'cegep',
 'prets-bourses-temps-plein',
 'aide-financiere',
 'enfance',
 'etudier-quebec',
 'refugi

In [847]:
import os, shutil
from pathlib import Path
from slugify import slugify

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../03-corpus/2-sous-corpus/'

#Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags:
    file_name =  acteur + "_" + slugify(str(tag).strip('\n'))

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    try:
        data = csv[csv["Address"].str.match('.*[\-\_\/]'+tag+'.*')]
        data.to_csv(folder_path + file_name + ".csv") #encoding='ISO-8859-1'
    except Exception as e:
        print(tag, str(e))
                    

In [848]:
with open(folder_path + acteur + '_sous-corpus.txt', 'w') as f:
    for file in os.listdir(folder_path):
        f.write(file + '\n')

_____________________________________________________________________________________________________________________

### **Étape d'indexation manuelle**

_____________________________________________________________________________________________________________________

Après avoir manuellement indexé

In [849]:
from pandas import read_csv
filepath = '../03-corpus/2-sous-corpus/' + acteur + '.csv'
with open(filepath, 'r', encoding = 'utf-8') as f:
    csv = read_csv(f, sep=';')
    
csv = csv[csv['Sous-corpus'] != 'x']
csv = csv[csv['Sous-corpus'] != 'X']

csv

,Dossier,Sous-corpus
1,quebec_sante_acces-information.csv,Ressources
3,quebec_sante_activites-sportives-et-de-plein-a...,Habitudes de vie
4,quebec_sante_agriculture-environnement-et-ress...,Environnement
5,quebec_sante_agriculture-pecheries-alimentatio...,Habitudes de vie
6,quebec_sante_agriculture-urbaine.csv,Agriculture
7,quebec_sante_agriculture.csv,Agriculture
12,quebec_sante_alimentation.csv,Habitudes de vie
17,quebec_sante_conseils-et-prevention.csv,Prévention
19,quebec_sante_coronavirus-2019.csv,COVID-19
23,quebec_sante_depistage-et-offre-de-tests-de-po...,Dépistage


In [850]:
# tags = list(set([tag for tag in csv['oui/non'].tolist()]))
tags = list(set([tag for tag in csv['Sous-corpus'].tolist()]))

tags

['Zoonoses',
 'Conditions sociales',
 'Dépistage',
 'ITSS',
 'Santé',
 'Problèmes de santé courants',
 'Vaccination',
 'Médecine légale',
 'Agriculture',
 'Urgence',
 'Santé mentale',
 'Prévention',
 'Soins et services',
 'Immigration',
 'Habitudes de vie',
 'COVID-19',
 'Ressources',
 'Environnement',
 'Toxicologie',
 'Périnatalité',
 'Enfance / Jeunesse',
 'Santé animale',
 'Violence',
 'Enfances / Jeunesse',
 'Ministère',
 'Hygiène et salubrité',
 'Carrières']

In [851]:
regroupements = {tag : [] for tag in tags}
for tag in tags:
        tuples = csv[csv['Sous-corpus'] == tag]
        liste = [str(t).strip('\n') for t in tuples["Dossier"].tolist()]
        regroupements[tag] = liste


In [852]:
regroupements

{'Zoonoses': ['quebec_sante_maladies-animales.csv'],
 'Conditions sociales': ['quebec_sante_refugies-demandeurs-asile.csv'],
 'Dépistage': ['quebec_sante_depistage-et-offre-de-tests-de-porteur.csv'],
 'ITSS': ['quebec_sante_itss.csv'],
 'Santé': ['quebec_sante_sante.csv'],
 'Problèmes de santé courants': ['quebec_sante_problemes-de-sante.csv'],
 'Vaccination': ['quebec_sante_vaccination.csv'],
 'Médecine légale': ['quebec_sante_expertise-judiciaire-medecine-legale.csv'],
 'Agriculture': ['quebec_sante_agriculture-urbaine.csv',
  'quebec_sante_agriculture.csv'],
 'Urgence': ['quebec_sante_securite-situations-urgence.csv',
  'quebec_sante_urgences-sinistres-risques-naturels.csv'],
 'Santé mentale': ['quebec_sante_informer-sur-troubles-mentaux.csv',
  'quebec_sante_sante-mentale.csv',
  'quebec_sante_troubles-mentaux.csv'],
 'Prévention': ['quebec_sante_conseils-et-prevention.csv'],
 'Soins et services': ['quebec_sante_organisation-des-services.csv',
  'quebec_sante_systeme-et-services-de

In [854]:
import pandas
from slugify import slugify

# Créer de nouveaux CSV contenant les données regroupées selon le tag qui leur est associé
path='../03-corpus/2-sous-corpus/'

for tag in regroupements:

    # Aller chercher les éléments correspondant au sous-corpus dans le csv principal et les copier dans un nouveau CSV 
    dfs = []
    path='../03-corpus/2-sous-corpus/'
    for doc in regroupements[tag]:
        with open(path + doc, 'r', encoding='utf-8') as f:
            csv = read_csv(f)
            dfs.append(csv)

    # Concatenate all data into one DataFrame
    try:
        path='../03-corpus/2-data/1-fr/' + acteur +'/'
        file_path = path + acteur + '_'  + slugify(str(tag)) + '.csv'
        big_frame = pandas.concat(dfs, ignore_index=True)
        big_frame['Sous-corpus'] = tag
        big_frame['Corpus'] = acteur
        big_frame = big_frame[["Corpus", "Sous-corpus",	"Address",	"Title", "Type",	"text"]]
        big_frame.to_csv(file_path)
        print(tag + "\t" + str(len(big_frame)))

    except Exception as e:
        print(tag, str(e))

Zoonoses	9
Conditions sociales	10
Dépistage	9
ITSS	11
Santé	258
Problèmes de santé courants	59
Vaccination	13
Médecine légale	12
Agriculture	120
Urgence	117
Santé mentale	67
Prévention	67
Soins et services	48
Immigration	50
Habitudes de vie	71
COVID-19	36
Ressources	46
Environnement	223
Toxicologie	17
Périnatalité	13
Enfance / Jeunesse	20
Santé animale	19
Violence	17
Enfances / Jeunesse	94
Ministère	172
Hygiène et salubrité	31
Carrières	77
